In [29]:
import pandas as pd
import wikipediaapi as api
import numpy as np
import matplotlib.pylab as plt
import wikipedia as wikip
from collections import Counter
import random as rd
import json
import requests
from statistics import mean
from bs4 import BeautifulSoup as bs
import urllib3
import re
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

## Ersten 30 Links

In [30]:
def getOrderedLinks(wikipage):
    #try:
    http = urllib3.PoolManager()
    response = http.request("GET", wikipage)
    soup = bs(response.data)
    soup = soup.find(id="mw-content-text")
    link_list = soup.find_all("a",href=re.compile("/wiki/"))
    return link_list
    #except:
        #print(wikipage)
        
def filterLinks(soup):
    reg = re.compile("^(?!/wiki/Datei:).+")
    reg2 = re.compile("^(?!/wiki/Spezial:).+")
    reg3 = re.compile("^/wiki")
    filteredList = [e["title"] for e in soup if reg.match(e["href"]) and reg2.match(e["href"]) and reg3.match(e["href"])]
    return filteredList

def cutList(titleList, length):
    result = ((titleList[:length]))
    return list(dict.fromkeys(result))



In [31]:
wiki = api.Wikipedia('Datenanalyse', 'de')

## Speichern der Listen als Textdateien

In [32]:
def save(liste, dateipfad):
    with open(dateipfad, 'w', encoding='utf-8') as datei:
        json.dump(liste, datei, ensure_ascii=False)

In [33]:
def WikiPageView(name, granularity, date_start, date_end):
    #YYYYMMDD
    # Calling monthly page views of each species 
    address = "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/de.wikipedia.org/all-access/user/" + name + "/" + granularity + "/" + date_start + "/" + date_end
    # Personal username for identification for the Wikipedia API
    headers = {'User-Agent': 'marek.seipel@web.de'}
    
    resp = requests.get(address, headers=headers)
    details = resp.json()
    
    return details

## Alle Einzelseiten der Kategorie Zweiter Weltkrieg

In [34]:
ww2 = []
def print_categorymembers(categorymembers, liste, namesp, level=0, max_level=10):
        for c in categorymembers.values():
            if c.ns == namesp and level < max_level:
                #print("%s: %s (ns: %d)" % ("*" * (level + 1), c.title, c.ns))
                liste.append(c.title)
        return liste

kategorie1 = wiki.page("Kategorie:Zweiter Weltkrieg")
ww2 = print_categorymembers(kategorie1.categorymembers, ww2, 0)
print(len(ww2))
ww2

20


['Zweiter Weltkrieg',
 'Chronologie des Zweiten Weltkrieges',
 'Literaturliste zum Thema Zweiter Weltkrieg',
 'Alliierte',
 'Beutekunst (Zweiter Weltkrieg)',
 'Blitzkrieg',
 'Wagen von Compiègne',
 'Deutsche Kriegsgefangene in den Vereinigten Staaten',
 'Die Dritte Welt im Zweiten Weltkrieg',
 'Internierungspraxis in Frankreich (1939–1945)',
 'Konzentrationslager (historischer Begriff)',
 'Kriegführende Staaten im Zweiten Weltkrieg',
 'Kriegswirtschaft',
 'Middle East Relief and Refugee Administration',
 'Nissenhütte',
 'Nothilfe- und Wiederaufbauverwaltung der Vereinten Nationen',
 'Osttruppen',
 'Vermisste Person',
 'Westmächte',
 'Zweiter Dreißigjähriger Krieg']

## Alle Unterkategorien der Kategorie Zweiter Weltkrieg

In [35]:
subcats = []
def print_subcategorymembers(categorymembers, subcatlist, namesp, level=0, max_level=10):
        for c in categorymembers.values():
            if c.ns != namesp and level < max_level:
                #print("%s: %s (ns: %d)" % ("*" * (level + 1), c.title, c.ns))
                subcatlist.append(c.title)
        return subcatlist

print_subcategorymembers(kategorie1.categorymembers, subcats, 0)
del subcats[0]
print(subcats)

['Kategorie:Zweiter Weltkrieg nach Schauplatz', 'Kategorie:Zweiter Weltkrieg nach Staat', 'Kategorie:Achsenmächte', 'Kategorie:Deutsche Besetzung Europas 1939–1945', 'Kategorie:Endphase des Zweiten Weltkriegs', 'Kategorie:Folgen des Zweiten Weltkriegs', 'Kategorie:Führerhauptquartier', 'Kategorie:Gedenken an den Zweiten Weltkrieg', 'Kategorie:Historiographie (Zweiter Weltkrieg)', 'Kategorie:Holocaust', 'Kategorie:Kriegsgefangenschaft und Internierung (Zweiter Weltkrieg)', 'Kategorie:Kriegswirtschaft im Zweiten Weltkrieg', 'Kategorie:Medien im Zweiten Weltkrieg', 'Kategorie:Militärgeschichte (Zweiter Weltkrieg)', 'Kategorie:Militärtechnik (Zweiter Weltkrieg)', 'Kategorie:Museum über den Zweiten Weltkrieg', 'Kategorie:Nachrichtendienste im Zweiten Weltkrieg', 'Kategorie:Orden und Ehrenzeichen (Zweiter Weltkrieg)', 'Kategorie:Person im Zweiten Weltkrieg', 'Kategorie:Politik (Zweiter Weltkrieg)', 'Kategorie:Propaganda im Zweiten Weltkrieg', 'Kategorie:Rattenlinien', 'Kategorie:Strategische

## Alle Seiten der Unterkategorien und Kategorie Zweiter Weltkrieg

In [36]:
memberliste = []
def getAllPages(subcategories, pagelist):
    for c in subcategories:
        subcat = wiki.page(c)
        print_categorymembers(subcat.categorymembers, pagelist, 0)
    return pagelist

memberliste = getAllPages(subcats,memberliste)
memberliste = list(set(memberliste))
print(memberliste)
print(len(memberliste))

['Lascaris War Rooms', 'Schlacht um Posen', 'Adolf-Hitler-Panzerprogramm', 'Moskauer Deklaration', 'Reichsamt für Wirtschaftsausbau', 'Heimkehrer', 'Tötungsanstalt Hartheim', 'Teilkapitulation der Wehrmacht für Nordwestdeutschland, Dänemark und die Niederlande', 'Brückenkopf Elsass', 'Verwundeten- und Flüchtlingstransporte über die Ostsee 1945', 'Luftangriff auf Wallhausen (Helme)', 'AN-M76', 'Lord Haw-Haw', 'Advanced Landing Grounds – Serie AAF', 'Japanisch-Sowjetischer Neutralitätspakt', 'Shiraume Gakutotai', 'USS Tarawa (LHA-1)', 'Vermögensabgabe (Lastenausgleichsgesetz)', 'Führerhauptquartier Wolfsschanze', 'Baltenauslieferung', 'USS Cape St. George (CG-71)', 'Braune Mappe', 'Commissariat Général aux Questions Juives', 'Luftnachrichtenabteilung 350', 'Museum der Ardennenschlacht Clervaux', 'Polnischer Enigma-Nachbau', 'Reserve-Polizei-Bataillon 101', 'Erinnerungsmedaille für die spanischen Freiwilligen im Kampf gegen den Bolschewismus', 'Deutsch-lettischer Nichtangriffspakt', 'Erin

## 100 zufällig gewählte Artikel der Kategorie Zweiter Weltkrieg und dessen Unterkategorien

In [37]:
randomlist = rd.sample(memberliste,100)
print(randomlist)
save(randomlist, "random_ww2")

['VENONA-Projekt', 'Orzeł-Zwischenfall', 'Lettere di condannati a morte della Resistenza Europea', 'Lorientschild', 'Ehrenschild des XI. Flieger-Korps', 'RSHA-Referat IV D 4', 'Belgische Annexionspläne nach dem Zweiten Weltkrieg', 'Plakette des Luftgaues Charkow', 'Massaker von Pawłokoma', 'Four Freedoms', 'Aktion Silberstreif', 'Kreuz von Monte Cassino', 'Vorgeschichte des Zweiten Weltkrieges im Pazifikraum', 'Aalborg Verteidigungs- und Garnisonsmuseum', 'Schlacht um Schloss Itter', 'Judenfrei', 'Deutsch-Litauischer-Nichtangriffspakt', 'Le Souvenir français', 'Seefliegerhorst Aalborg', 'Erdkampfabzeichen der Luftwaffe', 'Funkmessstörsender', 'Eiserne Ehrenplakette des Luftgaues XII', 'Tscheljabinski Traktorny Sawod', 'Wehrmachtbefehlshaber', 'Gruppe Manouchian', 'Sonderstandesamt Bad Arolsen', 'Bachem Ba 349', 'Stunde Null', 'Janusz Gumkowski', 'Mr. Blabbermouth!', 'Operation Epsilon', 'Interalliierte Kriegsschulden', 'Totaler Krieg', 'Mourir pour Dantzig?', 'Pariser Friedenskonferenz

## Alle Backlinks der 100 zufälligen Artikel der Kategorie Zweiter Weltkrieg und dessen Unterkategorien

In [38]:
backlinks_ww2 = []
def getBacklinks(liste, backlinks):
    for p in liste:
        backlinks.extend(wiki.page(p).backlinks)
    return backlinks

#backlinks_ww2 = list(set(getBacklinks(randomlist,backlinks_ww2)))
#save(backlinks_ww2, 'backlinks_ww2')
backlinks_ww2

[]

## 100 zufällige Einzelseiten von Tieren

In [39]:
tiere = ['Löwe', 'Pinguin', 'Schlange', 'Giraffe', 'Eichhörnchen', 'Delfin', 'Eule', 'Ameise', 'Känguru', 'Krokodil',
         'Elefant', 'Tiger', 'Panda', 'Koala', 'Flamingo', 'Hai', 'Adler', 'Frosch', 'Gecko', 'Gorilla',
         'Nashorn', 'Schildkröte', 'Schnecke', 'Kamel', 'Gepard', 'Wombat', 'Ameisenbär', 'Kuh', 'Maulwurf',
         'Fledermaus', 'Kolibri', 'Seestern', 'Erdmännchen', 'Papagei', 'Pferd', 'Schmetterling', 'Wale', 'Esel',
         'Krebs', 'Hummel', 'Faultier', 'Aal', 'Hamster', 'Pinguin', 'Elch', 'Hase', 'Mücke',
         'Eidechse', 'Quokka', 'Stachelschwein', 'Tintenfisch', 'Känguru', 'Schimpanse', 'Igel', 'Hirsch', 'Oktopus',
         'Uhu', 'Papagei', 'Dachs', 'Hai', 'Dromedar', 'Nasenaffe', 'Kakadu', 'Fasan', 'Murmeltier',
         'Makrele', 'Frettchen', 'Delfin', 'Lama', 'Schnabeltier', 'Eichhörnchen', 'Walhai', 'Stinktier', 'Affenbrotbaum',
         'Giraffe', 'Biber', 'Büffel', 'Pfeilgiftfrosch', 'Schwein', 'Wüstenfuchs', 'Humboldt-Pinguin', 'Pudel', 'Bambi',
         'Gnu', 'Möwe', 'Geier', 'Wombat', 'Zebra', 'Seelöwe', 'Husky', 'Otter', 'Koala',
         'Ameisenbär', 'Seepferdchen', 'Krake', 'Erdkröte', 'Bisamratte', 'Flusspferd', 'Hammerhai', 'Oktopus', 'Hund']


save(tiere, 'random_tiere')

## Alle Backlinks der 100 zufälligen Tierartikel

In [40]:
backlinks_tiere = []
backlinks_tiere = list(set(getBacklinks(tiere, backlinks_tiere)))
print(len(backlinks_tiere))
print(backlinks_tiere)
save(backlinks_tiere,'backlinks_tiere')

14023
['Ornithorhynchus', 'Platypus (Münze)', 'Vereinigtes Königreich', 'Neues von Pettersson und Findus', 'Wikipedia:Ruhrgebiet/Archiv/1', 'Lumpy-skin-Krankheit', 'Apulischer Fischteller (Heidelberg U 23)', 'Yucatan-Hörnchen', 'Tierlautbezeichnungen', 'Milchleistung', 'Benutzer:Regiomontanus/Sandkasten', 'Dagbatitherium', 'Nationalpark Salonga', 'Twyfelfontein', 'Wikipedia:Fragen zur Wikipedia/Archiv/2008/Woche 40', 'Siling (Mythologie)', 'Mountain-Zebra-Nationalpark', 'Murambya-See', 'Pekingese', 'Wikipedia:Vandalismusmeldung/Archiv/2006/11/16', 'Vercors', 'Johann von Brienne', 'White River First Nation', 'Anschießer', 'Wikipedia:WikiReader/Eulen', 'Nordby Kirke (Samsø)', 'Apophis', 'Benutzer:Formatierer/potentielle Tippfehler', 'Fiordland-Nationalpark', 'Benutzer:Denis Barthel/Bücher/Bradypus', 'Benutzer Diskussion:80.81.13.47', 'Reserva Ecológica de Buenos Aires', 'Coffin Rock', 'Rhadinovirus', 'Klippschliefer', 'Begriffe der Harry-Potter-Romane', 'Italienische Euromünzen', 'Benutz

## Alle Einzelseiten der Kategorie New York

In [41]:
ny = []
kategorie_ny = wiki.page("Kategorie:New York City")
ny = print_categorymembers(kategorie_ny.categorymembers, ny, 0)
ny

['New York City',
 'I Love New York (Werbekampagne)',
 'Siegel von New York City']

## Alle Unterkategorien der Kategorie New York

In [42]:
subny = []
print_subcategorymembers(kategorie_ny.categorymembers, subny, 0)
del subny[0]
print(subny)


['Kategorie:Bronx', 'Kategorie:Brooklyn', 'Kategorie:Manhattan', 'Kategorie:Queens', 'Kategorie:Staten Island', 'Kategorie:Bildung in New York City', 'Kategorie:Geographie (New York City)', 'Kategorie:Geschichte von New York City', 'Kategorie:Kultur (New York City)', 'Kategorie:Medien (New York City)', 'Kategorie:Organisation (New York City)', 'Kategorie:Organisation (New York City) als Thema', 'Kategorie:Person (New York City)', 'Kategorie:Planen und Bauen (New York City)', 'Kategorie:Politik (New York City)', 'Kategorie:Religion (New York City)', 'Kategorie:Sport (New York City)', 'Kategorie:Veranstaltung in New York City', 'Kategorie:Veranstaltungsstätte in New York City', 'Kategorie:Veranstaltungsstätte in New York City als Thema', 'Kategorie:Verkehr (New York City)', 'Kategorie:Wirtschaft (New York City)', 'Kategorie:Wissenschaft in New York City']


## Alle Seiten der Unterkategorien und der Kategorie New York

In [43]:
ny = getAllPages(subny, ny)
ny = list(set(ny))
print(len(ny))
ny

1390


['Stable Gallery',
 'Aeolian Hall (New York)',
 'Lowline (New York City)',
 'Jack Lait',
 'Sid Luckman',
 'The Joshua Light Show',
 'Masa Kamaguchi',
 'American Geographical Society',
 'Larry Davis (Mörder)',
 'Manhattan Transfer (Roman)',
 'Hank Azaria',
 'Trude Schiff-Löwenstein',
 'Walter Hinrichsen',
 'Society of American Historians',
 'Fleischer Studios',
 'Pen + Brush',
 'Daily News',
 'Charlotte Mason (Philanthropin)',
 'Gipfeltreffen in New York (1988)',
 'Olga Knopf',
 'John Gavin Nolan',
 'Bert Stern',
 'Alan Merrill',
 'Bloomberg Businessweek',
 'Charles Anderson Dana',
 'Cedar Walton',
 'Carol Sudhalter',
 'Memorial Sloan Kettering Cancer Center',
 'Death Mask',
 'Bruce Green',
 'Maxwell Perkins',
 'Miller Army Airfield',
 'Center for Constitutional Rights',
 'Aram Avakian',
 'Winter Garden Theatre',
 'Donald Trump Jr.',
 'West End Cafe',
 'Lesbian Sex Mafia',
 'Brian G. Hutton',
 'John Hill Hewitt',
 'Sommer-Paralympics 1984/Medaillenspiegel/New York',
 'Connie’s Inn',
 'P

## 100 zufällige Artikel aus der Kategorie New York und dessen Unterkategorien

In [44]:
randomny = rd.sample(ny,100)
print(randomny)
save(randomny, 'randomny')

['AMORC', 'Edward Jay Epstein', 'Dieter Finke (Bildhauer)', 'New York City Taxi Cabs', 'Ludwig Wronkow', 'Carnegie Hall', 'Maurice Wertheim', 'Studs Terkel', 'Oscar Noriega', 'National Urban League', 'James H. Torrens', 'Tan Dun', 'Margarita Tupitsyn', 'I Love New York (Werbekampagne)', 'Alfred Gwynne Vanderbilt', 'Institute for Architecture and Urban Studies', 'Eric Hoffer', 'Alan Alda', 'Alfred E. Smith Memorial Foundation Dinner', 'Mafia-Camorra-Krieg', 'Samuel Barlow (Komponist)', 'Nora Ephron', 'Liam Aiken', 'Irving Plaza', 'Bob Belden', 'Hildegard Bachert', 'Charles Norris', 'Bronx Zoo', 'Tara Donovan', 'Sarai Sherman', 'Café Society', 'Gotham Awards 2023', 'Rudolf Stingel', 'Peter Evans (Trompeter)', 'William Safire', 'Evan A. Lottman', 'Manhattan (Schiff, 1932)', 'Mercedes Ruehl', 'Charles Rosen (Musiker)', 'Bernhard Goetz', 'MTV Video Music Awards 2020', 'Draft Riots', 'Helen Frankenthaler', 'Environ', 'Paula Cooper Gallery', 'Frankie Genaro', 'Broadway (Theater)', 'The Tin Pa

## Alle Backlinks der 100 zufälligen New York-Artikel

In [45]:
nyBacklinks = []
nyBacklinks = list(set(getBacklinks(randomny,nyBacklinks)))
print(nyBacklinks)
save(nyBacklinks, 'backlinks_ny')

['Olympische Sommerspiele 1920/Teilnehmer (Norwegen)', 'Diskussion:Adriaen van der Donck', 'Randy Harrison', 'Wikipedia:Löschkandidaten/28. Juni 2008', 'Miles Smiles', 'Victor Baravalle', 'The Healer (Dr.-Lonnie-Smith-Album)', '375 Pearl Street', 'Portal:Jazz/Neue Artikel/2011', 'All in My Mind', 'Dash Snow', 'Dimitra Arliss', 'Edward Platt', 'Knight’s Foundry and Shops', 'Wildlife Conservation Society', 'Hermann Josef Abs', 'Walt Weiskopf', 'Wikipedia:Wartung/Beispiele', 'Peggy Cass', 'Fellow Travelers', 'The Women – Von großen und kleinen Affären', 'Bill Hayes (Schauspieler)', 'Melba Moore', 'Kabarett der Namenlosen', 'Jonathan Scott Hartley', 'Christopher Rich', 'Freddy Bienstock', 'Louis F. Bishop', 'Manhattan Island (New York)', 'Marjorie Lord', 'Loch Lomond', 'Santa Fe Opera', 'Richard Keese', 'Nekrolog 1935', 'Lorenzo Semple junior', 'O. Z. Whitehead', 'Skinamarink', 'Benutzer:KUI/Labor/Berlin-Charlottenburg-Nord', 'Church of Satan', '50 West Street', 'Cinderella (Broadway-Music

In [46]:
len(nyBacklinks)

14106

## Alle Einzelseiten der Kategorie Nahostkonflikt

In [47]:
nahostliste = []
kategorie4 = wiki.page("Kategorie:Nahostkonflikt")
nahostliste = print_categorymembers(kategorie4.categorymembers, nahostliste, 0)
print(len(nahostliste))
nahostliste

132


['Nahostkonflikt',
 'Arabisch-israelische Kriege',
 'Israelisch-palästinensischer Konflikt',
 'Ababil',
 'Abnutzungskrieg',
 'Affäre um deutsche Raketenexperten in Ägypten',
 'Streitkräfte Ägyptens',
 'Ägyptisches Heer',
 'Anglo-amerikanisches Untersuchungskomitee',
 'Anlage 1391',
 'Anschlag auf die Multinationalen Streitkräfte im Libanon am 23. Oktober 1983',
 'Arabische Friedensinitiative',
 'Artillerieangriff auf Kana',
 'Auspicia quaedam',
 'Bar-Lev-Linie',
 'Battir',
 'Be’er-Scheva-Gefängnis',
 'Beziehungen zwischen dem Iran und den Vereinigten Staaten',
 "Bil'in",
 'Blaue Linie (Libanon)',
 'Boycott, Divestment and Sanctions',
 'Bund Arabischer Republiken',
 'Bündnis für Gerechtigkeit zwischen Israelis und Palästinensern',
 'Chiyam',
 'Dis:orient',
 'EcoPeace',
 'Farbe Rot',
 'Friedensprozess im Nahen Osten',
 'Front der Standhaftigkeit',
 'Genfer Initiative',
 'Genfer Nahostkonferenz',
 'Geschichte des Staates Israel',
 'Gezielte Tötung',
 'Golanhöhen',
 'Grüne Linie',
 'Gusch 

## 100 zufällige Artikel der Kategorie Nahostkonflikt

In [48]:
random_nahost = rd.sample(nahostliste, 100)
save(random_nahost, 'random_nahost')
random_nahost

['Farbe Rot',
 'Vertreibung von Juden aus arabischen und islamischen Ländern',
 'Israelische Friedensdiplomatie',
 'Resolution 46/86 der UN-Generalversammlung',
 'Resolution 651 des UN-Sicherheitsrates',
 'Republik Haifa',
 'Schovrim Schtika',
 'Anlage 1391',
 'Israelische Verteidigungsstreitkräfte',
 'Sykes-Picot-Abkommen',
 'Muqataa',
 'Resolution 659 des UN-Sicherheitsrates',
 'Irakische Streitkräfte',
 'Bündnis für Gerechtigkeit zwischen Israelis und Palästinensern',
 'Operation Nördlicher Schutzschild',
 'Liste der Gouvernements der Palästinensischen Autonomiebehörde',
 'Lynchmord von Ramallah',
 "Bil'in",
 'Gusch Etzion',
 'Golanhöhen',
 'Palästinensisches Flüchtlingsproblem',
 'Operation Litani',
 'Bund Arabischer Republiken',
 'Streitkräfte Ägyptens',
 'Resolution 279 des UN-Sicherheitsrates',
 'Artillerieangriff auf Kana',
 'Resolution 803 des UN-Sicherheitsrates',
 'Palästinakrieg',
 'Auspicia quaedam',
 'Iranisch-israelische Beziehungen',
 'Naksa',
 'Society of St. Yves',
 '

## Alle Backlinks der 100 zufälligen Artikel der Kategorie Nahostkonflikt

In [49]:
backlinks_nahost = []
backlinks_nahost = list(set(getBacklinks(random_nahost, backlinks_nahost)))
print(len(backlinks_nahost))
save(backlinks_nahost, 'backlinks_nahost')
backlinks_nahost

5463


['Raghib an-Naschaschibi',
 'Benutzer:NormanK',
 'Maki (historische Partei)',
 'Liste der Biografien/Lao',
 'Communium interpretes dolorum',
 'Geschichte der palästinensischen Autonomiegebiete',
 'Streitkräfte Indiens',
 'Robert Ford (General)',
 'Syrisch-türkische Beziehungen',
 'Jarqonbahn',
 'Klippschliefer',
 'Mannschaftsaufstellungen der Schacholympiade 1990',
 'Staatsangehörigkeit der Vereinigten Arabischen Emirate',
 'OT-62 TOPAS',
 'Benutzer:Afroehlicher/Fünfte Republik',
 'Titularbistum Caesarea Philippi',
 'Wikipedia Diskussion:WikiProjekt Wartungsbausteine/Mangelartikel der Woche',
 'CRAMRA',
 'Hikmat Abu Zayd',
 'Resolution 862 des UN-Sicherheitsrates',
 'Wikipedia:Vandalismusmeldung/Archiv/2021/05/16',
 'Road map',
 'Benutzer:Se4598/BA NamedRefs/complex/8',
 'Niv Sultan',
 'Benutzer:Bluemel1/September 1995',
 'Vier sephardische Synagogen (Jerusalem)',
 'Raad Madschid al-Hamdani',
 'Lynn Hastings',
 'Idit Silman',
 'Diskussion:Nimrod (Burg)',
 'Wikipedia:Vandalismusmeldung/

## Umwandlung Wikipedia-Tabelle von verstorbenen Prominenten in Dataframe

In [50]:
monate = ['Februar','März','April', 'Mai', 'Juni', 'Juli','August','September','Oktober','November']

#Nekrolog für Januar 2023
wikiurl="https://de.wikipedia.org/wiki/Nekrolog_Januar_2023"
table_class="wikitable sortable zebra jquery-tablesorter"
response=requests.get(wikiurl)

soup = bs(response.text, 'html.parser')
indiatable=soup.find('table',{'class':"wikitable"})

df=pd.read_html(str(indiatable))
df_gesamt=pd.DataFrame(df[0])


for monat in monate:
    wikiurl="https://de.wikipedia.org/wiki/Nekrolog_" + monat + "_2023"
    table_class="wikitable sortable zebra jquery-tablesorter"
    response=requests.get(wikiurl)

    soup = bs(response.text, 'html.parser')
    indiatable=soup.find('table',{'class':"wikitable"})

    df=pd.read_html(str(indiatable))
    df_monat=pd.DataFrame(df[0])
    df_gesamt = pd.concat([df_gesamt, df_monat], join="inner", ignore_index= True)
    
df_gesamt


C:\Users\yusuf\AppData\Local\Temp\ipykernel_18044\2966956431.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df=pd.read_html(str(indiatable))
C:\Users\yusuf\AppData\Local\Temp\ipykernel_18044\2966956431.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df=pd.read_html(str(indiatable))
C:\Users\yusuf\AppData\Local\Temp\ipykernel_18044\2966956431.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df=pd.read_html(str(indiatable))
C:\Users\yusuf\AppData\Local\Temp\ipykernel_18044\2966956431.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

,Tag,Name,"Beruf, bekannt als",Alter,Beleg
0,31. Januar,Ferdinand Anders,österreichischer Historiker und Völkerkundler,92,[1]
1,31. Januar,Cleonice Berardinelli,brasilianische Literaturwissenschaftlerin,106,[2]
2,31. Januar,Joyce Dopkeen,US-amerikanische Pressefotografin,80,[3]
3,31. Januar,David Durenberger,US-amerikanischer Politiker,88,[4]
4,31. Januar,Craig William Kallendorf,US-amerikanischer Philologe,68,[5]
...,...,...,...,...,...
5218,1. November,Peter Tarnoff,US-amerikanischer Diplomat,86,[376]
5219,1. November,Vittorio Vergeat,italienischer Musiker,72,[377]
5220,1. November,Peter White,US-amerikanischer Schauspieler,86,[378]
5221,1. November,Steffen Winkler,deutscher Museologe und Ethnograph,71,[379]


## 100 zufällige, verstorbene Promis

In [51]:
promis = []

for ind in df_gesamt.index:
    page = wiki.page(df_gesamt['Name'][ind])
    if page.exists() == True:
        promis.append(df_gesamt['Name'][ind])
print(len(promis))
promis

4089


['Ferdinand Anders',
 'Cleonice Berardinelli',
 'David Durenberger',
 'Craig William Kallendorf',
 'Bernd Karwofsky',
 'Kim Young-hee',
 'Jan Kudra',
 'Sylvia Lamsfuß',
 'Henrik Nordbrandt',
 'Kadriye Nurmambet',
 'Luigi L. Pasinetti',
 'Edmond Roudnitska',
 'Barbara Scofield',
 'Charlie Thomas',
 'Michael Wellenhofer',
 'Wiktor Agejew',
 'Alan Bates',
 'Martha Eibl',
 'Donald Hess',
 'Bobby Hull',
 'Charly Loubet',
 'Ann McLaughlin Korologos',
 'Gerald Mortag',
 'Gerhard Ohneis',
 'Félix Sienra',
 'Charles Silverstein',
 'Peter Bahn',
 'Jürgen Fabian',
 'Ross Gillespie',
 'Dora Lardelli',
 'Heddy Lester',
 'Hazel McCallion',
 'Gerhard Moehring',
 'Dmytro Pawlytschko',
 'Gerhard Raab',
 'Gerhard Remus',
 'George R. Robertson',
 'Roger Schank',
 'Kyle Smaine',
 'Will Steffen',
 'Gero Storjohann',
 'Barrett Strong',
 'Piotr Waśko',
 'Annie Wersching',
 'Gerhard Woitzik',
 'Josef Zenzmaier',
 'Phillip Coles',
 'Hans Otto Eglau',
 'Jane F. Gardner',
 'Karlheinz Götz',
 'Anne-Barb Hertkorn'

In [52]:
random_promis = rd.sample(promis,100)
save(random_promis, 'random_promis')
random_promis

['Wladimir Skulatschow',
 'Franz Böni',
 'Wiktorija Amelina',
 'Paolo Magnani',
 'Pearry Reginald Teo',
 'Donn Cambern',
 'Heidy Forster',
 'James G. Watt',
 'Raymond Ackerman',
 'Siegfried Heinrich',
 'Ruud Geels',
 'Gunther Wawrik',
 'Ursel Sieber',
 'Andreas Hanske',
 'Hans Modrow',
 'Hein Spreckelsen',
 'Sergio Ottolina',
 'George V. Chilingar',
 'Zeno Giacomuzzi',
 'Anneliese Zänsler',
 'Kurt Schellnack',
 'Wiktor Pikaisen',
 'Hugo Partsch',
 'Werner Gifhorn',
 'Peter W. von Weymarn',
 'Brigitte Tilmann',
 'Helen Williams',
 'Murray G. Hall',
 'Ron Cephas Jones',
 'Marcel Duriez',
 'Ingo Bockler',
 'George Maharis',
 'Sławomir Maciejowski',
 'Ingrid Olef',
 'Matthew Perry',
 'Ulrich Schoknecht',
 'Hans Gerlach',
 'Philippe Borer',
 'Piers Haggard',
 'Wolfgang Becker',
 'Nancy Van de Vate',
 'Jesús Guzmán',
 'Luigi Berlinguer',
 'Robert Day',
 'Tom Courtney',
 'Cécile Cordon',
 'James Herbain',
 'Gerhard Schlegel',
 'Malika El Aroud',
 'Werner Hölzenbein',
 'Ferdinand Stracke',
 'G

## Alle Backlinks der 100 zufälligen, verstorbenen Promis

In [53]:
backlinks_promis = []
backlinks_promis = getBacklinks(random_promis,backlinks_promis)
print(len(backlinks_promis))
save(backlinks_promis,'backlinks_promis')
backlinks_promis

2691


['Benutzer:P. W. Siebert/Erstellte Weiterleitungen',
 '1952',
 'Conrad-Ferdinand-Meyer-Preis',
 'Liste deutschsprachiger Schriftsteller/B',
 'Literaturpreis der Stadt Bremen',
 'Benutzer:Michelangelo/Autorenliste',
 'Liste Schweizer Schriftsteller',
 'Böni',
 'Grosser Schillerpreis',
 'Schweizerische Schillerstiftung',
 'Portal:Deutsche Literatur/Archiv/2005-2006',
 'Ammann Verlag',
 'Ingeborg-Bachmann-Preis 1979',
 'Liste von Persönlichkeiten der Stadt Winterthur',
 'Diskussion:Conrad-Ferdinand-Meyer-Preis',
 'Benutzer:Schelmentraum',
 'Portal:Literatur/Perlentaucher/B',
 'Benutzer:FA2010/Killy',
 'Liste der Nachlässe in der Schweizerischen Nationalbibliothek',
 'Benutzer:WolfgangRieger/Killy Lemmaliste',
 'Literaturjahr 1952',
 'Simone Meier',
 'Liste der Biografien/Boni',
 'Liste der Bände der edition suhrkamp',
 'Literaturjahr 2023',
 'Liste auf dem Friedhof Rosenberg bestatteter Persönlichkeiten',
 'Diskussion:Liste auf dem Friedhof Rosenberg bestatteter Persönlichkeiten',
 'Nekro